In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import shutil
from sklearn.model_selection import train_test_split
import time

In [3]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
base_dir = "/kaggle/input/realwaste/realwaste-main/RealWaste"
for folder in os.listdir(base_dir):
    count = len(os.listdir(os.path.join(base_dir, folder)))
    print(f"{folder}: {count} images")

Metal: 790 images
Glass: 420 images
Paper: 500 images
Vegetation: 436 images
Cardboard: 461 images
Textile Trash: 318 images
Food Organics: 411 images
Plastic: 921 images
Miscellaneous Trash: 495 images


In [13]:
import os
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import random
import numpy as np
import torch
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

# Configuration
BATCH_SIZE = 64
IMG_SIZE = (224, 224)
DATA_DIR = "/kaggle/input/realwaste/realwaste-main/RealWaste"

In [6]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [8]:
import splitfolders  # install with: pip install split-folders

input_folder = base_dir
split_dir = "/kaggle/working/RealWaste_split"

splitfolders.ratio(input_folder, output=split_dir, seed=42, ratio=(.7, .15, .15))


Copying files: 0 files [00:00, ? files/s]
Copying files: 1 files [00:01,  1.98s/ files]
Copying files: 83 files [00:02, 55.61 files/s]
Copying files: 162 files [00:02, 119.90 files/s]
Copying files: 239 files [00:02, 192.03 files/s]
Copying files: 319 files [00:02, 275.11 files/s]
Copying files: 398 files [00:02, 359.16 files/s]
Copying files: 478 files [00:02, 442.07 files/s]
Copying files: 560 files [00:02, 521.55 files/s]
Copying files: 643 files [00:02, 592.08 files/s]
Copying files: 723 files [00:02, 629.97 files/s]
Copying files: 801 files [00:04, 161.12 files/s]
Copying files: 857 files [00:04, 174.99 files/s]
Copying files: 903 files [00:04, 172.94 files/s]
Copying files: 941 files [00:04, 174.27 files/s]
Copying files: 979 files [00:05, 198.32 files/s]
Copying files: 1058 files [00:05, 282.91 files/s]
Copying files: 1137 files [00:05, 368.64 files/s]
Copying files: 1211 files [00:06, 130.45 files/s]
Copying files: 1252 files [00:06, 135.98 files/s]
Copying files: 1286 files [

In [10]:
# Count images in each subfolder
for split in ['train', 'val', 'test']:
    split_path = os.path.join(split_dir, split)
    print(f"\n{split.upper()} SET")
    total = 0
    for cls in os.listdir(split_path):
        cls_path = os.path.join(split_path, cls)
        count = len(os.listdir(cls_path))

        total += count
        print(f"  {cls}: {count} images")
    print(f" Total {split}: {total} images")


TRAIN SET
  Glass: 294 images
  Food Organics: 287 images
  Metal: 553 images
  Miscellaneous Trash: 346 images
  Plastic: 644 images
  Cardboard: 322 images
  Textile Trash: 222 images
  Paper: 350 images
  Vegetation: 305 images
 Total train: 3323 images

VAL SET
  Glass: 63 images
  Food Organics: 61 images
  Metal: 118 images
  Miscellaneous Trash: 74 images
  Plastic: 138 images
  Cardboard: 69 images
  Textile Trash: 47 images
  Paper: 75 images
  Vegetation: 65 images
 Total val: 710 images

TEST SET
  Glass: 63 images
  Food Organics: 63 images
  Metal: 119 images
  Miscellaneous Trash: 75 images
  Plastic: 139 images
  Cardboard: 70 images
  Textile Trash: 49 images
  Paper: 75 images
  Vegetation: 66 images
 Total test: 719 images


In [15]:
import os
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms
import numpy as np # Already imported later, but useful here
from tqdm import tqdm # For progress bars


train_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=45),
        transforms.RandomResizedCrop(size=IMG_SIZE[0], scale=(0.6, 0.9)),
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor()
])

val_test_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder("/kaggle/working/RealWaste_split/train", transform=train_transforms)
val_dataset   = datasets.ImageFolder("/kaggle/working/RealWaste_split/val", transform=val_test_transforms)
test_dataset  = datasets.ImageFolder("/kaggle/working/RealWaste_split/test", transform=val_test_transforms)

# Get number of classes and device early
num_classes = len(train_dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"Number of classes: {num_classes}")

Using device: cuda
Number of classes: 9


In [25]:
from torch.utils.data import ConcatDataset

duplication_factor = 3
train_dataset_augmented = ConcatDataset([train_dataset] * duplication_factor)

In [29]:
# Assuming train_dataset, val_dataset, test_dataset are defined from Block 1

# Extract targets manually
targets = []
for dataset in train_dataset_augmented.datasets:  # ConcatDataset.datasets is a list
    if hasattr(dataset, 'targets'):
        targets.extend(dataset.targets)
    else:
        # fallback if custom dataset
        for i in range(len(dataset)):
            _, label = dataset[i]
            targets.append(label)

# Convert to numpy
import numpy as np
targets = np.array(targets)

# Class counts
class_counts = np.bincount(targets)
print("Class counts:", class_counts)

base_dataset = train_dataset_augmented.datasets[0]  # first dataset in the concat list
classes = base_dataset.classes

# Compute class weights (inverse of frequency)
class_weights_tensor = 1. / torch.tensor(class_counts, dtype=torch.float)

# Ensure sample_weights are floats for the sampler
sample_weights = [class_weights_tensor[label].item() for label in targets]

print("Class counts per category:")
for cls, count in zip(classes, class_counts):
    print(f"  {cls:15s}: {count}")

print("\n Class weights (inverse of frequency):")
for cls, w in zip(classes, class_weights_tensor):
    print(f"  {cls:15s}: {w:.6f}")

Class counts: [ 966  861  882 1659 1038 1050 1932  666  915]
Class counts per category:
  Cardboard      : 966
  Food Organics  : 861
  Glass          : 882
  Metal          : 1659
  Miscellaneous Trash: 1038
  Paper          : 1050
  Plastic        : 1932
  Textile Trash  : 666
  Vegetation     : 915

 Class weights (inverse of frequency):
  Cardboard      : 0.001035
  Food Organics  : 0.001161
  Glass          : 0.001134
  Metal          : 0.000603
  Miscellaneous Trash: 0.000963
  Paper          : 0.000952
  Plastic        : 0.000518
  Textile Trash  : 0.001502
  Vegetation     : 0.001093


In [30]:
# Define a consistent BATCH_SIZE
BATCH_SIZE = 32 # Increased batch size

train_loader = DataLoader(train_dataset_augmented, batch_size=BATCH_SIZE, shuffle=True) # ADDED: num_workers
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False) # ADDED: val_loader
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False) # ADDED: test_loader


print(f"\n WeightedRandomSampler created successfully!")
print(f" Total samples in epoch: {len(sample_weights)}")
print(f" Training Batch size: {train_loader.batch_size}")
print(f" Validation Batch size: {val_loader.batch_size}")
print(f" Total training batches per epoch: {len(train_loader)}")


 WeightedRandomSampler created successfully!
 Total samples in epoch: 9969
 Training Batch size: 32
 Validation Batch size: 32
 Total training batches per epoch: 312


In [47]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        # ======== TRAINING ========
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * correct_train / total_train
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        # ======== VALIDATION ========
        model.eval()
        val_loss_total = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss_total += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        epoch_val_loss = val_loss_total / len(val_loader.dataset)
        epoch_val_acc = 100 * correct_val / total_val
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}: "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%, "
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")

    return train_losses, val_losses, train_accuracies, val_accuracies


In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=9):
        super(CNNModel, self).__init__()

        # Blocco convoluzionale base
        self.conv_block1 = self._create_conv_block(3, 64)  # 64 filtri
        self.conv_block2 = self._create_conv_block(64, 128)  # 128 filtri
        self.conv_block3 = self._create_conv_block(128, 256)  # 256 filtri
        self.conv_block4 = self._create_conv_block(256, 512)  # 512 filtri
        self.conv_block5 = self._create_conv_block(512, 512)  # 512 filtri aggiuntivi

        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        # Strati completamente connessi
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.3)

    def _create_conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)

        # Global Average Pooling
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten

        # Strati densi
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
# Istanza del modello
model = CNNModel(num_classes=9)

model_name = "Custom CNN"

# Configurazione del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Conversione dei pesi al dispositivo
weights = torch.tensor(class_weights.copy(), dtype=torch.float32).to(device)  # Personalizza i pesi
criterion = nn.CrossEntropyLoss(weight=weights)

# Modello sul dispositivo
model = model.to(device)

# Ottimizzatore
learning_rate = 1e-5
weight_decay = 1e-2
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate,
    weight_decay=weight_decay
)

# Scheduler del learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Numero di epoche
num_epochs = 20

# Training del modello
train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs
)

# Plot delle metriche di training e validazione
plot_training_validation_metrics(
    train_losses=train_losses,
    val_losses=val_losses,
    train_accuracies=train_accuracies,
    val_accuracies=val_accuracies
)

Using device: cuda
Epoch 1/20: Train Loss: 1.5532, Train Acc: 46.37%, Val Loss: 1.3023, Val Acc: 55.77%
Epoch 2/20: Train Loss: 1.1399, Train Acc: 60.20%, Val Loss: 1.0935, Val Acc: 61.27%
Epoch 3/20: Train Loss: 0.9598, Train Acc: 66.60%, Val Loss: 0.9461, Val Acc: 66.76%
Epoch 4/20: Train Loss: 0.8635, Train Acc: 69.58%, Val Loss: 0.8761, Val Acc: 71.41%
Epoch 5/20: Train Loss: 0.7945, Train Acc: 71.65%, Val Loss: 0.7934, Val Acc: 72.54%
Epoch 6/20: Train Loss: 0.7255, Train Acc: 74.20%, Val Loss: 0.7646, Val Acc: 73.38%
Epoch 7/20: Train Loss: 0.6803, Train Acc: 75.65%, Val Loss: 0.7058, Val Acc: 74.37%
Epoch 8/20: Train Loss: 0.6291, Train Acc: 77.19%, Val Loss: 0.7353, Val Acc: 73.24%


In [ ]:
# Save the model
model_save_path = f"/kaggle/working/trained_{model_name}_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")
